# Zadanie 5


Celem ćwiczenia jest implementacja perceptronu wielowarstwowego oraz wybranego algorytmu optymalizacji gradientowej z algorytmem propagacji wstecznej.

Następnie należy wytrenować perceptron wielowarstwowy do klasyfikacji zbioru danych [MNIST](http://yann.lecun.com/exdb/mnist/). Zbiór MNIST dostępny jest w pakiecie `scikit-learn`.

Punktacja:
1. Implementacja propagacji do przodu (`forward`) [1 pkt]
2. Implementacja wstecznej propagacji (zademonstrowana na bramce XOR) (`backward`) [2 pkt]
3. Przeprowadzenie eksperymentów na zbiorze MNIST, w tym:
    1. Porównanie co najmniej dwóch architektur sieci [1 pkt]
    2. Przetestowanie każdej architektury na conajmniej 3 ziarnach [1 pkt]
    3. Wnioski 1.[5 pkt]
4. Jakość kodu 0.[5 pkt]

Polecane źródła - teoria + intuicja:
1. [Karpathy, CS231n Winter 2016: Lecture 4: Backpropagation, Neural Networks 1](https://www.youtube.com/watch?v=i94OvYb6noo&ab_channel=AndrejKarpathy)
2. [3 Blude one Brown, Backpropagation calculus | Chapter 4, Deep learning
](https://www.youtube.com/watch?v=tIeHLnjs5U8&t=4s&ab_channel=3Blue1Brown)


In [642]:
from abc import abstractmethod, ABC
from typing import List
import numpy as np

import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

In [643]:
# from sklearn.metrics import accuracy_score,classification_report
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import svm

mnist = fetch_openml('mnist_784')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [644]:
print(mnist.DESCR)
mnist.target.shape

'''
image= mnist.data.to_numpy()
plt.subplot(431)
plt.imshow((image[0].reshape(28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
plt.subplot(432)
plt.imshow(image[1].reshape(28,28), cmap=plt.cm.gray_r, interpolation= 'nearest')
plt.subplot(433)
plt.imshow(image[3].reshape(28,28), cmap=plt.cm.gray_r, interpolation= 'nearest')
plt.subplot(434)
plt.imshow(image[4].reshape(28,28), cmap=plt.cm.gray_r, interpolation= 'nearest')
plt.subplot(435)
plt.imshow(image[5].reshape(28,28), cmap=plt.cm.gray_r, interpolation= 'nearest')
plt.subplot(436)
plt.imshow(image[6].reshape(28,28), cmap=plt.cm.gray_r, interpolation= 'nearest')
'''

index_number= np.random.permutation(70000)
x1,y1=mnist.data.loc[index_number],mnist.target.loc[index_number]
x1.reset_index(drop=True,inplace=True)
y1.reset_index(drop=True,inplace=True)
x_train , x_test = x1[:55000], x1[55000:]
y_train , y_test = y1[:55000], y1[55000:]



**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [645]:
mnist.data

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [646]:
mnist.target.shape

(70000,)

In [647]:
x_train_array = x_train.to_numpy()
x_train_array = x_train_array/255
# The original pixel values range from 0 to 255. So, we divided them by 255 to get them into the range of 0.0 to 1.0.
# https://www.google.com/search?client=opera&q=how+mnist+dataset+is+normalized&sourceid=opera&ie=UTF-8&oe=UTF-8
type(x_train_array[0])

numpy.ndarray

In [648]:
y_train_array = y_train.to_numpy(dtype=int)
def normalize_to_vector(a:int):
    arr = np.zeros(10)
    arr[a] +=1
    return arr

y_train_array = np.array([normalize_to_vector(number) for number in y_train_array])
print(y_train_array)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [649]:
class Layer(ABC):
    """Basic building block of the Neural Network"""

    def __init__(self) -> None:
        self._learning_rate = 0.01

    @abstractmethod
    def forward(self, x:np.ndarray)->np.ndarray:
        """Forward propagation of x through layer"""
        pass

    @abstractmethod
    def backward(self, output_error_derivative) ->np.ndarray:
        """Backward propagation of output_error_derivative through layer"""
        pass

    @property
    def learning_rate(self):
        return self._learning_rate

    @learning_rate.setter
    def learning_rate(self, learning_rate):
        assert learning_rate < 1, f"Given learning_rate={learning_rate} is larger than 1"
        assert learning_rate > 0, f"Given learning_rate={learning_rate} is smaller than 0"
        self._learning_rate = learning_rate


class FullyConnected(Layer):
    def __init__(self, input_size:int, output_size:int) -> None:
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.rand(self.input_size, self.output_size) - 0.5
        self.biases = np.random.rand(1, self.output_size) - 0.5


    def forward(self, x_input:np.ndarray)->np.ndarray:
        self.input = x_input.reshape(1, self.input_size)
        #print(self.input)
        #print("FullyConnected(Layer) - forward -> input: ", self.input.shape)
        self.output = np.dot(self.input, self.weights) + self.biases
        #print("FullyConnected(Layer) - forward -> output: ", self.output.shape)
        #print("self.weights", self.weights.shape)
        #print("self.weights", self.weights)
        #print("Input: ",self.input,"\n", "Output", self.output, "\n weights", self.weights)
        return self.output

    def backward(self, output_error_derivative)->np.ndarray:

        #print("FullyConnected(Layer) - backward -> output_derivative: ", output_error_derivative.shape)

        # output_error_derivative = dE/dz
        # input_error_derivative: dE/dx = dE/dz * dz/dx (=weights)
        input_error_derivative = np.dot(output_error_derivative, self.weights.T)
        # weight_error_derivative: dE/dw = dE/dz * dz/dw
        weight_error_derivative =  np.dot(self.input.T, output_error_derivative)
        # bias_error_derivative: dE/db = dE/db = dE/dz * dz/db (=1) = dE/dz
        bias_error_derivative = output_error_derivative

        # gradient descent
        #print("weight_error_derivative.shape", weight_error_derivative.shape)
        #print("self.weights", self.weights.shape)
        #print("weight_error_derivative * self.learning_rate", weight_error_derivative * self.learning_rate)
        self.weights -= weight_error_derivative * self.learning_rate
        self.biases -= bias_error_derivative * self.learning_rate

        #print("weights", self.weights)

        #print("FullyConnected(Layer) - backward -> input_derivative: ", input_error_derivative.shape)
        return input_error_derivative

class Tanh(Layer):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, x:np.ndarray)->np.ndarray:
        self.input = x
        #print("Tanh(Layer) - forward -> input: ", self.input.shape)
        self.output = np.tanh(x)
        #print("Tanh(Layer)) - forward -> output: ", self.output.shape)
        #print("Tahn self.output", self.output)
        return self.output

    def backward(self, output_error_derivative)->np.ndarray:
        #print("Tanh(Layer) - backward -> output_derivative: ", output_error_derivative.shape)
        input_error_derivative = (1 - np.tanh(self.input)**2) * output_error_derivative
        #print("Tanh(Layer) - backward -> input_derivative: ", input_error_derivative.shape)

        #print("input_error_derivative", input_error_derivative)

        return input_error_derivative

class Loss:
    def __init__(self, loss_function:callable, loss_function_derivative:callable)->None:
        self.loss_function = loss_function
        self.loss_function_derivative = loss_function_derivative

    def loss(self, x:np.ndarray)->np.ndarray:
        """Loss function for a particular x"""
        return self.loss_function(x)

    def loss_derivative(self, x:np.ndarray)->np.ndarray:
        """Loss function derivative for a particular x and y"""
        return self.loss_function_derivative(x)

class Network:
    def __init__(self, layers:List[Layer], learning_rate:float)->None:
        self.layers = layers
        self.learning_rate = learning_rate

    def compile(self, loss:Loss)->None:
        """Define the loss function and loss function derivative"""
        self.loss = loss

    def __call__(self, x_input:np.ndarray) -> np.ndarray:
        """Forward propagation of x through all layers"""
        output = x_input
        for layer in self.layers:
            output = layer.forward(output)

        '''loss_function = lambda x: (x-y_train_array[0])**2
        loss_derivative = lambda x: 2*(x-y_train_array[0])
        self.compile(Loss(loss_function, loss_derivative))
        print("self.loss.loss(output)",self.loss.loss(output))
        print("y_train_array[0]", y_train_array[0])'''
        return output

    def fit(self,
            x_train:np.ndarray,
            y_train:np.ndarray,
            epochs:int,
            learning_rate:float,
            verbose:int=0)->None:
        """Fit the network to the training data"""
        for _ in range(epochs):
            for training, x_train_sample in enumerate(x_train):

                loss_function = lambda x: (x - y_train[training])**2
                loss_derivative = lambda x: 2*(x - y_train[training])
                self.compile(Loss(loss_function, loss_derivative))

                output = x_train_sample
                for layer in self.layers:
                    layer.learning_rate = learning_rate
                    output = layer.forward(output)

                #print("Loss:", self.loss.loss(output))

                input = self.loss.loss_derivative(output)

                for layer in reversed(self.layers):
                    input = layer.backward(input)


In [650]:
x_train_xor = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train_xor = np.array([[[1.,0.]], [[0.,1.]], [[0.,1.]], [[1.,0.]]])

In [651]:
net = Network([FullyConnected(2,5), Tanh(), FullyConnected(5,2), Tanh()], 0.05)#, FullyConnected(3,1), Tanh()], 0.1)
net.fit(x_train_xor, y_train_xor, 1000, 0.01)
net.__call__(x_train_xor[0])

array([[0.69957354, 0.04463043]])

In [652]:
x_train_array_new = x_train_array[:5000]
y_train_array_new = y_train_array[:5000]

In [653]:
n = Network([FullyConnected(28**2, 100), Tanh(), FullyConnected(100, 50), Tanh(), FullyConnected(50, 10), Tanh()], 0.1)

n.fit(x_train_array_new, y_train_array_new, 10, 0.009)

print(n.__call__(x_train_array_new[0]))





[[ 0.01293167  0.9434586  -0.11343197 -0.05912619  0.05899302 -0.13029401
   0.13608491 -0.0091659   0.09144389 -0.09022397]]


# Eksperymenty

# Wnioski

Bibliografia:



1.   https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65
2. Sztuczna inteligencja dla inżynierów. Metody ogólne
3. https://www.youtube.com/watch?v=i94OvYb6noo
4. https://www.youtube.com/watch?v=tIeHLnjs5U8
5. https://medium.com/@vaibhav1403/fully-connected-layer-f13275337c7c
6. https://mahendra-choudhary.medium.com/mnist-handwritten-digits-recognition-using-scikit-learn-36161768d10e
7. https://www.baeldung.com/cs/epoch-neural-networks
https://en.wikipedia.org/wiki/MNIST_database